# CIFAR10 CNN Pytorch

## 1. Library Call & Setting

In [ ]:
from tqdm.notebook import tqdm
import numpy as np
import random
import warnings
warnings.filterwarnings('ignore')

import torch
from torchvision import datasets, transforms

GPU

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Randomness Control

In [ ]:
seed = 42
if device == 'cuda':
    torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## 2. Data Load

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train = datasets.CIFAR10(root='./data',
                         train=True,
                         download=True,
                         transform=transform)

test = datasets.CIFAR10(root='./data',
                        train=False,
                        download=True,
                        transform=transform)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
train

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [ ]:
test

Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

## 3. Model

Model Definition

In [ ]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 6, 5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2))
        
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(6, 16, 5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2))
        
        self.fc1 = torch.nn.Linear(16 * 5 * 5, 120, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)

        self.fc2 = torch.nn.Linear(120, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [ ]:
model = CNN().to(device)

Hyper-Parameter

In [ ]:
batch_size = 20
epochs = 10
learning_rate = 0.001

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss().to(device)

## 4. Training

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=2)

In [ ]:
for epoch in (range(epochs)):
    avg_loss = 0
    for X_train, y_train in tqdm(train_loader):
        X_train = X_train.to(device)
        y_train = y_train.to(device)

        optimizer.zero_grad()
        pred = model(X_train)
        loss = criterion(pred, y_train)
        loss.backward()
        optimizer.step()
        
        avg_loss += loss / len(train_loader)

    print(f"[{epoch+1:>2}/{epochs}] Loss: {avg_loss:.6f}")

  0%|          | 0/2500 [00:00<?, ?it/s]

[ 1/10] Loss: 1.492567


  0%|          | 0/2500 [00:00<?, ?it/s]

[ 2/10] Loss: 1.248227


  0%|          | 0/2500 [00:00<?, ?it/s]

...

[ 8/10] Loss: 1.056027


  0%|          | 0/2500 [00:00<?, ?it/s]

[ 9/10] Loss: 1.045779


  0%|          | 0/2500 [00:00<?, ?it/s]

[10/10] Loss: 1.037991


## 5. Prediction

In [ ]:
test_loader = torch.utils.data.DataLoader(test,
                                         batch_size=batch_size,
                                         shuffle=False,
                                         num_workers=2)

with torch.no_grad():
    total = 0
    correct = 0
    for X_test, y_test in test_loader:    
        X_test=X_test.to(device)
        y_test=y_test.to(device)

        pred_ = model(X_test)
        total += y_test.size(0)
        correct += (torch.argmax(pred_, 1) == y_test).sum().item()

    print(f"Accuracy: {correct * 100 / total:.2f}%")

Accuracy: 61.08%
